In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import norm
from scipy.spatial import KDTree
from scipy import sparse
import pandas as pd
import numpy as np
import re
import json

In [42]:
import sqlite3

conn = sqlite3.connect("scraped_data.db")

df = pd.read_sql_query("SELECT Preprocessed_Long_Text, Title, URL FROM reviews", conn)
df = df.head(10)

print(df)

                              Preprocessed_Long_Text  \
0  space conquest game penny include good war boa...   
1  give elden ring year ago incredible game raise...   
2  editor note review take fresh look stardew val...   
3  patiently wait sideline vr reason couch flail ...   
4  long game come memorable exciting fresh write ...   
5  ask want sequel legend zelda breath wild enemy...   
6  light high quality remake resident evil releas...   
7  ask favorite roguelike answer confidently unab...   
8  decade original release metroid prime problem ...   
9  seven dwarf set mountainhome establish new col...   

                                              Title  \
0                            Arcs Board Game Review   
1      Elden Ring: Shadow of the Erdtree DLC Review   
2                      Stardew Valley Review - 2024   
3                           Asgard’s Wrath 2 Review   
4                            Baldur's Gate 3 Review   
5  The Legend of Zelda: Tears of the Kingdom Review  

In [43]:
corpus = df['Preprocessed_Long_Text'].tolist()

v = TfidfVectorizer()

vec = v.fit_transform(corpus)


#print(v.get_feature_names_out())

In [44]:
print(vec.shape)
#print(v.vocabulary_)
#print(v.get_feature_names_out())
#print(pd.DataFrame(vec.toarray(), columns=v.get_feature_names_out()))

(10, 3496)


In [45]:
#einzelne Wörter oder Vektoren inspizieren 
test = pd.DataFrame(vec.toarray(), columns=v.get_feature_names_out())
print(test["war"])


0    0.016064
1    0.015815
2    0.012226
3    0.000000
4    0.000000
5    0.008932
6    0.039591
7    0.000000
8    0.000000
9    0.008854
Name: war, dtype: float64


In [46]:
# Eine Suchanfrage mit dem Korpus vergleichen 

query = ["war empire"]  # auch preprocessen!
query_vec = v.transform(query)  # Wichtig: transform(), nicht fit_transform()

scores = cosine_similarity(query_vec, vec)
print(scores)


[[0.02744769 0.00925576 0.02088949 0.         0.         0.00522776
  0.02317147 0.01355248 0.         0.00518203]]


In [69]:
# Suchmaschine

def search(query, top_k=3):
    # Preprocessing falls nötig (hier angenommen, dass der Query-Text sauber ist)
    query_vec = v.transform([query])

    # Kosinus-Ähnlichkeit berechnen
    similarities = cosine_similarity(query_vec, vec).flatten()

    # Ergebnisse sortieren (nach Ähnlichkeit absteigend)
    ranked_indices = similarities.argsort()[::-1]

    print(ranked_indices)
    
    # Ausgabe der Top-Ergebnisse
    results = []
    for idx in ranked_indices[:top_k]:
        results.append({
            "doc_id": idx,
            "similarity": similarities[idx],
            "Title": df.iloc[idx]['Title'], 
            #"URL": df.iloc[idx]['URL']
        })
    
    return pd.DataFrame(results)

In [77]:
import spacy
from tqdm.notebook import tqdm

tqdm.pandas()


nlp = spacy.load("en_core_web_sm")

def queryPreprocessing(text): 
    if not isinstance(text, str):
        return []

    
    doc = nlp(text.lower()) # kleinbuchstaben
    tokens = [
            token.lemma_ # lemmatisierung
            for token in doc
            if token.is_alpha and not token.is_stop # zahlen entfernen und stopwörter
        ]
    string_list =" ".join(tokens)
    return string_list

In [82]:
# Suchmaschine testen 

query = "Resident has Evil"
query = queryPreprocessing(query)

print(query)
results = search(query)
print(results)

resident evil
[6 2 8 9 7 3 5 4 1 0]
   doc_id  similarity                            Title
0       6    0.371629           Resident Evil 4 Review
1       2    0.020252     Stardew Valley Review - 2024
2       8    0.016634  Metroid Prime Remastered Review


In [ ]:
# To-Do: Store Model so not needed to retrain

# To-Do: Implement KDTree